In [ ]:
"""
Task 2. 
    Input -
        Datasets_for_A1 folder needs to be in the current working 
        directory with the folder structure as 
        Datasets_for_A1/Regression/Dataset 2/<team number>/
        Replace <team number> with the actual number.
        e.g. Datasets_for_A1/Regression/Dataset 2/9/
    Output -
        Linear model for regression using polynomial basis functions for Dataset 2
        Regularization methods:
            1. No regularization
            2. Quadratic regularization
        Presentation of Results:
            • For Task2: Plots of the surfaces of the approximated function obtained 
            using training datasets of different sizes (50 and 200), for different 
            model complexities (degrees 2, 3 and 6) and for different values of λ. 
            The training data points need to be superposed on the surface.
            • Scatter plots with target output on x-axis and model output on y-axis 
            for the best performing model, for training data and test data.
            • Tables showing the 𝐸𝑅𝑀𝑆 on the training data, the validation
            data and the test data, for different models.
"""

import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

lambda_string = "λ"
column_names = [
    "Index",
    "Sample Size",
    "Degree",
    "λ",
    "Train Erms",
    "Validate Erms",
    "Test Erms",
]
table_index = 0
table_df = pd.DataFrame(columns=column_names)
fig = ""
ax = ""


def add_data_to_table(sample_size, M, lamda, train_erms, validate_erms, test_erms):
    row_data = {}
    global table_df
    global table_index
    table_index = table_index + 1
    row_data["Index"] = table_index
    row_data["Sample Size"] = sample_size
    row_data["Degree"] = M
    row_data["λ"] = lamda
    row_data["Train Erms"] = train_erms
    row_data["Validate Erms"] = validate_erms
    row_data["Test Erms"] = test_erms
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        table_df = pd.concat(
            [table_df, pd.DataFrame(row_data, index=[0])], ignore_index=True
        )


def print_table():
    print(table_df.to_string(index=False))


def factorial(n):
    ele = np.arange(1, n + 1)
    fact = np.prod(ele)

    return fact


def get_N(x):
    N = len(x)
    return N


def get_D(d, M):
    D = factorial(M + d) / (factorial(M) * factorial(d))

    return D


def get_meshgrid(start, stop, N):
    x1_range = np.linspace(start, stop, N)
    x2_range = np.linspace(start, stop, N)
    x1, x2 = np.meshgrid(x1_range, x2_range)

    return (x1, x2)


def get_identity_matrix(n):
    identity_matrix = np.eye(n)

    return identity_matrix


def get_small_phi(x1i, x2i, M):
    small_phi = []
    for m in range(M + 1):
        for n in range(M - m + 1):
            small_phi.append((x1i**m) * (x2i**n))
            # print(f'(x1^{m})*(x2^{n}) = {(x1i ** m) * (x2i ** n)}')
    # print("x[", each_feature, "] ", x, " basis function [", small_phi, "]")
    # print("==============")
    return small_phi


def get_big_phi(x, M, N):
    big_phi = []
    each_feature = 0
    x1 = x[0]
    x2 = x[1]

    for each_feature in range(N):
        x1i = x1[each_feature]
        x2i = x2[each_feature]
        small_phi = get_small_phi(x1i, x2i, M)
        big_phi.append(small_phi)
    np_big_phi = np.array(big_phi)
    # print("design matrix ", np_big_phi)
    return np_big_phi


def get_new_y(w, x1x2, N, M, D):
    x1 = x1x2[0]
    x2 = x1x2[1]

    # print("x1", x1)
    # print("x2", x2)

    new_y = []
    loop1 = 0

    for loop1 in range(N):
        # for loop1 in range(len(val_x)):
        x1i = x1[loop1]
        x2i = x2[loop1]
        small_phi = get_small_phi(x1i, x2i, M)
        # print("small phi ",small_phi)
        y = 0
        loop2 = 0
        for loop2 in range(D):
            wi = w[loop2]
            xi = small_phi[loop2]
            y = y + (wi * xi)
        new_y.append(y)
    # print("new_y", new_y)
    np_new_y = np.array(new_y)
    # print("new_y", np_new_y)
    return np_new_y


def get_erms(y, t):
    loop = 0
    total_data = len(y)
    # print("len y ", len(y), " len t ", len(t))
    sum = 0

    for loop in range(total_data):
        sum = sum + np.square(y[loop] - t[loop])
        # print("loop ", loop, " sum ", sum)

    error = sum / total_data

    erms = np.sqrt(error)

    return erms


def scatter_plot_2d(x, y, x_label, y_label, plot_title, plot_color):
    plt.scatter(x, y, color=plot_color, label=plot_title)
    plt.title(plot_title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    # plt.show()


def scatter_plot_3d(x, y, plot_title, plot_color):
    global ax
    scatter = ax.scatter(x[0], x[1], y, color=plot_color)
    ax.set_title(plot_title)
    ax.set_xlabel("x1")
    ax.set_ylabel("x2")
    ax.set_zlabel("y")
    ax.legend([scatter], ["Training Data"])
    # plt.show()


def line_plot(polyline, y, plot_label, plot_color):
    plt.plot(polyline, y, color=plot_color, label=plot_label)
    plt.legend()
    # plt.show()


def surface_plot(x1x2, y, plot_label, plot_color):
    global ax
    x1 = x1x2[0]
    x2 = x1x2[1]
    # print("x1",x1)
    # print("x2",x2)
    # print("y",y)
    surface = ax.plot_surface(x1, x2, y, color=plot_color, alpha=0.5)
    ax.legend([surface], [plot_label])
    # plt.show()


def plot_erms(n):
    global table_df
    last_n_rows = table_df.tail(n)
    # print(last_4_rows)
    degree_array = last_n_rows["Degree"].values
    lambda_array = last_n_rows["λ"].values
    train_erms_array = last_n_rows["Train Erms"].values
    validate_erms_array = last_n_rows["Validate Erms"].values
    test_erms_array = last_n_rows["Test Erms"].values

    plot_title = "Train Erms"
    scatter_plot_2d(degree_array, train_erms_array, "M", "ERMS", plot_title, "blue")
    line_plot(degree_array, train_erms_array, plot_title, "blue")
    plot_title = "Validation Erms"
    scatter_plot_2d(degree_array, validate_erms_array, "M", "ERMS", plot_title, "green")
    line_plot(degree_array, validate_erms_array, plot_title, "green")
    plot_title = "Test Erms"
    scatter_plot_2d(degree_array, test_erms_array, "M", "ERMS", plot_title, "red")
    line_plot(degree_array, test_erms_array, plot_title, "red")
    plot_title = "ERMS λ=" + str(lambda_array[0])
    plt.title(plot_title)
    plt.show()


def get_least_erms(n):
    global table_df
    last_n_rows = table_df.tail(n)
    sorted_rows = last_n_rows.sort_values(by="Test Erms")
    first_sorted_row_values = sorted_rows.iloc[0].values
    # print(last_n_rows)
    # print(first_sorted_row_values)
    return first_sorted_row_values


def plot_scatter(sample_size, M, lamda, train_x, train_y, d, test_x, test_y):
    D = get_D(d, M)

    N = get_N(train_y)
    w = my_polyfit(train_x, train_y, M, N, int(D), lamda)
    plot_title = "Train Data Scatter Plot"
    new_train_y = get_new_y(w, train_x, N, M, int(D))
    scatter_plot_2d(train_y, new_train_y, "Actual", "Predicted", plot_title, "blue")
    # plt.show()

    N = get_N(test_y)
    plot_title = "Test Data Scatter Plot"
    new_test_y = get_new_y(w, test_x, N, M, int(D))
    scatter_plot_2d(test_y, new_test_y, "Actual", "Predicted", plot_title, "red")

    plot_title = (
        "ScatterPlot - SampleSize="
        + str(sample_size)
        + " M="
        + str(M)
        + " λ="
        + str(lamda)
    )
    plt.title(plot_title)
    plt.show()


def my_pseudo_inverse(matrix):
    matrix_transpose = np.transpose(matrix)
    phi_trans_multi_phi = np.dot(matrix_transpose, matrix)
    phi_trans_multi_phi_inv = np.linalg.inv(phi_trans_multi_phi)
    phi_trans_multi_phi_inv_multi_phi_trans = np.dot(
        phi_trans_multi_phi_inv, matrix_transpose
    )

    return phi_trans_multi_phi_inv_multi_phi_trans


def my_pseudo_inverse_with_regularization(matrix, matrix_dimension, lamda):
    matrix_transpose = np.transpose(matrix)
    phi_trans_multi_phi = np.dot(matrix_transpose, matrix)
    Identity = get_identity_matrix(matrix_dimension)
    regularization_matrix = np.multiply(Identity, lamda)
    phi_trans_multi_phi_add_regularized = np.add(
        phi_trans_multi_phi, regularization_matrix
    )
    phi_trans_multi_phi_inv = np.linalg.inv(phi_trans_multi_phi_add_regularized)
    phi_trans_multi_phi_inv_multi_phi_trans = np.dot(
        phi_trans_multi_phi_inv, matrix_transpose
    )

    return phi_trans_multi_phi_inv_multi_phi_trans


def my_polyfit(x, y, M, N, D, lamda=0.0):
    big_phi = get_big_phi(x, M, N)

    if lamda == 0.0:
        # pseudo_inv_np_big_phi = np.linalg.pinv(np_big_phi)
        # print("pseudo inverse design matrix ", pseudo_inv_np_big_phi)
        my_pseudo_inv_np_big_phi = my_pseudo_inverse(big_phi)
        # print("my pseudo inverse design matrix ", my_pseudo_inv_np_big_phi)
        pseudo_inv_np_big_phi = my_pseudo_inv_np_big_phi
    else:
        # pseudo_inv_np_big_phi = np.linalg.inv(np_big_phi.T @ np_big_phi + lamda * np.eye(np_big_phi.shape[1])) @ np_big_phi.T
        # print("pseudo inverse design matrix ", pseudo_inv_np_big_phi)
        my_regularized_pseudo_inv_np_big_phi = my_pseudo_inverse_with_regularization(
            big_phi, D, lamda
        )
        # print("my regularized pseudo inverse design matrix ", my_regularized_pseudo_inv_np_big_phi)
        pseudo_inv_np_big_phi = my_regularized_pseudo_inv_np_big_phi

    w = np.dot(pseudo_inv_np_big_phi, y)
    # print("w ", w)

    return w


def my_poly1d(w, x1x2, N, M, D):
    new_y = get_new_y(w, x1x2, N, M, D)

    return new_y


def plot_graphs(train_x, train_y, d, title, val_x, val_y, test_x, test_y, lamda=0.0):
    degrees = [2, 3, 6]
    start = -5
    stop = 5
    # degrees = [6]
    # print(title)
    # column_names = ['Sample Size', 'Degree', 'λ', 'Train Erms', 'Validate Erms', 'Test Erms']
    # table_df = pd.DataFrame(columns=column_names)

    for M in degrees:
        N = get_N(train_y)
        D = get_D(d, M)
        global fig
        fig = plt.figure()
        global ax
        ax = fig.add_subplot(111, projection="3d")
        # print("N = ", N, " d = ", d, " M = ", M, " D = ", D)
        w = my_polyfit(train_x, train_y, M, N, int(D), lamda)
        sample_size = N
        x1x2 = get_meshgrid(start, stop, N)
        new_train_y = my_poly1d(w, x1x2, N, M, int(D))
        plot_title = title
        scatter_plot_3d(train_x, train_y, plot_title, "blue")
        surface_plot(x1x2, new_train_y, plot_title, "green")
        # print(x1x2)
        # print(new_train_y)
        new_train_y = get_new_y(w, train_x, N, M, int(D))
        train_erms = get_erms(new_train_y, train_y)
        # print("Erms(sample size = ", N, " degree = ", M, " lambda = ",regularization_coefficient,") train_erms = ",train_erms)

        N = get_N(val_y)
        x1x2 = get_meshgrid(start, stop, N)
        new_val_y = my_poly1d(w, x1x2, N, M, int(D))
        plot_title = "Val-9"
        surface_plot(x1x2, new_val_y, plot_title, "red")
        new_val_y = get_new_y(w, val_x, N, M, int(D))
        validate_erms = get_erms(new_val_y, val_y)
        # print("Erms(sample size = ", N, " degree = ", M, " lambda = ",regularization_coefficient,") validate_erms = ",validate_erms)

        N = get_N(test_y)
        x1x2 = get_meshgrid(start, stop, N)
        new_test_y = my_poly1d(w, x1x2, N, M, int(D))
        plot_title = "Test-9"
        surface_plot(x1x2, new_test_y, plot_title, "yellow")
        new_test_y = get_new_y(w, test_x, N, M, int(D))
        test_erms = get_erms(new_test_y, test_y)
        # print("Erms(sample size = ", N, " degree = ", M, " lambda = ",regularization_coefficient,") test_erms = ",test_erms)

        add_data_to_table(sample_size, M, lamda, train_erms, validate_erms, test_erms)

        plot_title = title + " M=" + str(M) + " λ=" + str(lamda)
        plt.title(plot_title)
        plt.show()

        plot_title = "Train"
        scatter_plot_2d(train_y, new_train_y, "Actual", "Predicted", plot_title, "blue")
        plot_title = "Test"
        scatter_plot_2d(test_y, new_test_y, "Actual", "Predicted", plot_title, "red")
        plot_title = (
            "SampleSize=" + str(sample_size) + " M=" + str(M) + " λ=" + str(lamda)
        )
        plt.title(plot_title)
        plt.show()

    plot_erms(len(degrees))
    min_erms = get_least_erms(len(degrees))
    plot_scatter(
        min_erms[1], min_erms[2], min_erms[3], train_x, train_y, d, test_x, test_y
    )


def main():
    folder_number = "9"
    current_directory = os.getcwd()
    # regression_dataset_2_path=current_directory+ "/Datasets_for_A1/Regression/Dataset 2/9/"
    regression_dataset_2_path = (
        "/home/dipendu/mtech/ml/ass1/Datasets_for_A1/Regression/Dataset 2/"
        + folder_number
        + "/"
    )

    regression_dataset_2_Train_Sample_1 = (
        regression_dataset_2_path + "train50_" + folder_number + ".csv"
    )
    df = pd.read_csv(regression_dataset_2_Train_Sample_1)
    train_x_11 = np.squeeze(np.asanyarray(df[["x1"]]))
    train_x_12 = np.squeeze(np.asanyarray(df[["x2"]]))
    train_y_1 = np.squeeze(np.asanyarray(df[["y"]]))
    data = list(zip(train_x_11, train_x_12, train_y_1))
    sorted_data = sorted(data, key=lambda x: x[0])
    train_x_11, train_x_12, train_y_1 = zip(*sorted_data)
    train_x_1 = []
    train_x_1.append(train_x_11)
    train_x_1.append(train_x_12)

    regression_dataset_2_Train_Sample_2 = (
        regression_dataset_2_path + "train200_" + folder_number + ".csv"
    )
    df = pd.read_csv(regression_dataset_2_Train_Sample_2)
    train_x_21 = np.squeeze(np.asanyarray(df[["x1"]]))
    train_x_22 = np.squeeze(np.asanyarray(df[["x2"]]))
    train_y_2 = np.squeeze(np.asanyarray(df[["y"]]))
    data = list(zip(train_x_21, train_x_22, train_y_2))
    sorted_data = sorted(data, key=lambda x: x[0])
    train_x_21, train_x_22, train_y_2 = zip(*sorted_data)
    train_x_2 = []
    train_x_2.append(train_x_21)
    train_x_2.append(train_x_22)

    regression_dataset_2_validation = (
        regression_dataset_2_path + "val_" + folder_number + ".csv"
    )
    df = pd.read_csv(regression_dataset_2_validation)
    val_x_1 = np.squeeze(np.asanyarray(df[["x1"]]))
    val_x_2 = np.squeeze(np.asanyarray(df[["x2"]]))
    val_y = np.squeeze(np.asanyarray(df[["y"]]))
    data = list(zip(val_x_1, val_x_2, val_y))
    sorted_data = sorted(data, key=lambda x: x[0])
    val_x_1, val_x_2, val_y = zip(*sorted_data)
    val_x = []
    val_x.append(val_x_1)
    val_x.append(val_x_2)

    regression_dataset_2_test = (
        regression_dataset_2_path + "test_" + folder_number + ".csv"
    )
    df = pd.read_csv(regression_dataset_2_test)
    test_x_1 = np.squeeze(np.asanyarray(df[["x1"]]))
    test_x_2 = np.squeeze(np.asanyarray(df[["x2"]]))
    test_y = np.squeeze(np.asanyarray(df[["y"]]))
    data = list(zip(test_x_1, test_x_2, test_y))
    sorted_data = sorted(data, key=lambda x: x[0])
    test_x_1, test_x_2, test_y = zip(*sorted_data)
    test_x = []
    test_x.append(test_x_1)
    test_x.append(test_x_2)

    regularization_coefficients = [0.0, 0.0001, 1e-9, 1e-18]
    # regularization_coefficients = [0.0]
    d = 2

    for lamda in regularization_coefficients:
        plot_graphs(
            train_x_1,
            train_y_1,
            d,
            "train50_" + folder_number,
            val_x,
            val_y,
            test_x,
            test_y,
            lamda,
        )

    for lamda in regularization_coefficients:
        plot_graphs(
            train_x_2,
            train_y_2,
            d,
            "train200_" + folder_number,
            val_x,
            val_y,
            test_x,
            test_y,
            lamda,
        )

    print_table()


if __name__ == "__main__":
    main()